In [ ]:
!nvidia-smi

Mon Apr  7 13:09:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!zip -r /content/my_model_ner_improved.zip /content/model_ner_improved
!zip -r /content/my_model_ner_saved.zip /content/model_ner_saved
!zip -r /content/my_results_improved.zip /content/results_improved
!pip install transformers datasets torch scikit-learn
!pip install peft bitsandbytes accelerate
!zip -r /content/my_model_few_shot.zip /content/model_few_shot
!zip -r /content/my_results_few_shot.zip /content/results_few_shot

In [ ]:
# Per PyTorch
import torch
print("GPU disponibile:", torch.cuda.is_available())
print("Nome GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nessuna GPU")


GPU disponibile: True
Nome GPU: Tesla T4


In [ ]:
# Per TensorFlow
import tensorflow as tf
print("Dispositivi GPU:", tf.config.list_physical_devices('GPU'))


Dispositivi GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import torch
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Sto usando:", device)

# Matrici grandi per testare uso GPU
x = torch.randn(5000, 5000).to(device)
y = torch.randn(5000, 5000).to(device)

start = time.time()
for _ in range(100):
    z = torch.mm(x, y)
torch.cuda.synchronize()  # Assicura che le operazioni finiscano
end = time.time()

print("Tempo GPU:", end - start)


Sto usando: cuda
Tempo GPU: 6.841125249862671


In [ ]:
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers datasets seqeval
!pip install -q datasets
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --upgrade

In [ ]:
def load_iob_file(path):
    sentences = []
    labels = []
    with open(path, encoding="utf-8") as f:
        tokens = []
        tags = []
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append(tokens)
                    labels.append(tags)
                    tokens, tags = [], []
            else:
                splits = line.split()
                tokens.append(splits[0])
                tags.append(splits[-1])
        if tokens:
            sentences.append(tokens)
            labels.append(tags)
    return sentences, labels

train_sents, train_labels = load_iob_file("train_iob_corretto.txt")
dev_sents, dev_labels = load_iob_file("dev_iob_corretto.txt")
test_sents, test_labels = load_iob_file("test_iob_corretto.txt")


In [ ]:
with open("classes.txt") as f:
    label_list = [line.strip() for line in f.readlines()]
label_list = ["O"] + label_list  # aggiungiamo la classe 'O'
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("expertai/LLaMAntino-3-SLIMER-IT")

def tokenize_and_align_labels(sentences, labels):
    tokenized_inputs = tokenizer(
        sentences,
        is_split_into_words=True,
        return_offsets_mapping=True,
        padding=True,
        truncation=True
    )
    aligned_labels = []
    for i, label in enumerate(labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned.append(-100)
            elif word_idx != previous_word_idx:
                aligned.append(label2id.get(label[word_idx], label2id["O"]))
            else:
                aligned.append(label2id.get(label[word_idx], label2id["O"]))
            previous_word_idx = word_idx
        aligned_labels.append(aligned)
    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset, DatasetDict

def load_iob_dataset(path):
    sentences, labels = [], []
    with open(path, encoding='utf-8') as f:
        tokens, tags = [], []
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append(tokens)
                    labels.append(tags)
                    tokens, tags = [], []
            else:
                splits = line.split()
                tokens.append(splits[0])
                tags.append(splits[-1])
        if tokens:
            sentences.append(tokens)
            labels.append(tags)
    return sentences, labels

train_tokens, train_tags = load_iob_dataset("/content/train_iob_corretto.txt")
dev_tokens, dev_tags = load_iob_dataset("/content/dev_iob_corretto.txt")
test_tokens, test_tags = load_iob_dataset("/content/test_iob_corretto.txt")


In [ ]:
train_ds = Dataset.from_dict({"tokens": train_tokens, "ner_tags": train_tags})
dev_ds = Dataset.from_dict({"tokens": dev_tokens, "ner_tags": dev_tags})
test_ds = Dataset.from_dict({"tokens": test_tokens, "ner_tags": test_tags})

dataset = DatasetDict({
    "train": train_ds,
    "validation": dev_ds,
    "test": test_ds
})


In [ ]:
def tokenize_and_align_labels(dataset):
    tokenized_inputs = tokenizer(
        dataset["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=256,
    )

    labels = []
    for i, label in enumerate(dataset["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)
            elif word_idx != previous_word_idx:
                aligned_labels.append(label2id.get(label[word_idx], 0))
            else:
                aligned_labels.append(label2id.get(label[word_idx], 0))
            previous_word_idx = word_idx
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Applica la trasformazione
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "expertai/LLaMAntino-3-SLIMER-IT"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)


config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at expertai/LLaMAntino-3-SLIMER-IT and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
torch.cuda.empty_cache()

claude piu veloce

primo codice

In [ ]:
# Librerie necessarie
!pip install bitsandbytes accelerate -q
!pip install -q datasets
!pip install seqeval -q
!pip install deepspeed
import gc
import os
import torch
import numpy as np
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForTokenClassification, BitsAndBytesConfig, DataCollatorForTokenClassification
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import Dataset, DatasetDict

# Configurazione CUDA per evitare problemi di frammentazione memoria
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Per essere sicuri di liberare tutta la memoria disponibile
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU disponibile: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Memoria GPU utilizzata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Opzionale: aggiungere una callback per monitorare meglio i tempi di addestramento
from transformers.integrations import TensorBoardCallback
import time
from transformers import TrainerCallback

class TimingCallback(TrainerCallback):
    def __init__(self):
        self.start_time = time.time()
        self.step_start_time = time.time()
        self.steps = 0

    def on_step_end(self, args, state, control, **kwargs):
        self.steps += 1
        if self.steps % 10 == 0:  # Stampa ogni 10 steps
            elapsed = time.time() - self.step_start_time
            steps_per_second = 10 / elapsed
            print(f"Step {state.global_step}: {steps_per_second:.2f} steps/s, {elapsed/10:.3f} s/step")
            self.step_start_time = time.time()

    def on_epoch_end(self, args, state, control, **kwargs):
        elapsed = time.time() - self.start_time
        print(f"Epoca {state.epoch} completata. Tempo totale: {elapsed/60:.2f} minuti")


# Define the label_list, label2id, and id2label
with open("classes.txt", "r", encoding="utf-8") as f:
    label_list = [line.strip() for line in f.readlines()]
#label_list = ["O"] + label_list  # aggiungiamo la classe 'O'
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

print(f"Etichette caricate: {label_list}")
print(f"Mapping etichette -> ID: {label2id}")

# Define the load_iob_dataset function
def load_iob_dataset(path):
    sentences, labels = [], []
    with open(path, encoding='utf-8') as f:
        tokens, tags = [], []
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append(tokens)
                    labels.append(tags)
                    tokens, tags = [], []
            else:
                splits = line.split()
                if len(splits) >= 2:  # Assicurati che ci siano almeno due colonne
                    tokens.append(splits[0])
                    tags.append(splits[-1])
        if tokens:
            sentences.append(tokens)
            labels.append(tags)
    return sentences, labels

# Carica il tokenizer
model_name = "expertai/LLaMAntino-3-SLIMER-IT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Funzione modificata per ridurre la lunghezza massima delle sequenze
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=32,              # Ridotto da 64 a 32
        return_tensors=None
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignora token speciali
            elif word_idx != previous_word_idx:
                # Solo il primo token di una parola ottiene l'etichetta
                if word_idx < len(label):
                    label_text = label[word_idx]
                else:
                    label_text = "O"  # Default a "O" per indici fuori limite
                label_id = label2id.get(label_text, label2id["O"])
                label_ids.append(label_id)
            else:
                # Token successivi della stessa parola
                label_ids.append(-100)  # Usiamo -100 per ignorare questi token nella loss

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Carica i dataset
print("Caricamento dei dataset...")
train_tokens, train_tags = load_iob_dataset("/content/train_sampled_iob_corretto.txt")
dev_tokens, dev_tags = load_iob_dataset("/content/dev_sampled_iob_corretto.txt")
test_tokens, test_tags = load_iob_dataset("/content/test_sampled_iob_corretto.txt")

# Analisi della lunghezza delle sequenze nel dataset
print("Analisi delle lunghezze delle sequenze:")
train_lengths = [len(seq) for seq in train_tokens]
print(f"Lunghezza media delle sequenze nel train set: {sum(train_lengths)/len(train_lengths):.1f} token")
print(f"Sequenza più lunga nel train set: {max(train_lengths)} token")
print(f"Percentile 95 delle lunghezze: {sorted(train_lengths)[int(len(train_lengths)*0.95)]} token")
print(f"Percentile 99 delle lunghezze: {sorted(train_lengths)[int(len(train_lengths)*0.99)]} token")

# Verifica la distribuzione delle etichette
print("Analisi della distribuzione delle etichette:")
all_tags = [tag for tag_list in train_tags for tag in tag_list]
unique_tags = set(all_tags)
print(f"Etichette uniche nel dataset: {unique_tags}")
tag_counts = {}
for tag in unique_tags:
    tag_counts[tag] = all_tags.count(tag)
    print(f"  - {tag}: {tag_counts[tag]} occorrenze")

# Calcolo pesi delle classi per class weighting
tag_weights = {}
total_tags = len(all_tags)
for tag in unique_tags:
    # Metodo inverso della frequenza
    tag_weights[tag] = total_tags / (tag_counts[tag] * len(unique_tags))
print(f"Pesi calcolati per le classi: {tag_weights}")

# Converti i pesi in un array per il modello
class_weights = []
for i in range(len(label_list)):
    label = id2label[i]
    if label in tag_weights:
        class_weights.append(tag_weights[label])
    else:
        # Gestisci etichette che potrebbero non essere presenti nei dati di training
        class_weights.append(1.0)

class_weights_tensor = torch.tensor(class_weights)
print(f"Tensor pesi classi: {class_weights_tensor}")

# Crea i dataset
train_ds = Dataset.from_dict({"tokens": train_tokens, "ner_tags": train_tags})
dev_ds = Dataset.from_dict({"tokens": dev_tokens, "ner_tags": dev_tags})
test_ds = Dataset.from_dict({"tokens": test_tokens, "ner_tags": test_tags})

dataset = DatasetDict({
    "train": train_ds,
    "validation": dev_ds,
    "test": test_ds
})

# Pre-processa i dataset
print("Pre-processing dei dataset...")
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["tokens", "ner_tags"]
)
print(f"Dataset processato: {len(tokenized_datasets['train'])} esempi nel train set")

# Verifica la struttura dei dati
print("Verifica della struttura dei dati:")
sample = tokenized_datasets["train"][0]
for key, value in sample.items():
    if isinstance(value, list):
        print(f"{key}: lista di lunghezza {len(value)}, tipo elementi: {type(value[0]).__name__}")
        if len(value) > 0 and isinstance(value[0], (int, float)):
            print(f"  - Esempio valori: {value[:5]}...")
    else:
        print(f"{key}: {type(value).__name__}")

# Configura la quantizzazione 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

print("Caricamento del modello con quantizzazione a 4-bit...")
# Crea il modello con quantizzazione a 4-bit e offload
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    device_map="auto",
    quantization_config=quantization_config,
    torch_dtype=torch.float16
)

# Monitora l'uso di memoria dopo il caricamento del modello
print(f"Memoria GPU dopo caricamento modello: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Prepara il modello per l'addestramento a 4-bit
model = prepare_model_for_kbit_training(model)

# Abilita checkpointing
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()
    print("Gradient checkpointing abilitato")

# Configurazione LoRA con moduli adatti al modello
# Trova quali moduli sono effettivamente nel modello
model_modules = [name for name, _ in model.named_modules()]
print("Moduli disponibili nel modello:")
for module in sorted(set([m.split('.')[-1] for m in model_modules if '.' in m]))[:15]:  # Mostra i nomi dei moduli unici
    print(f"  - {module}")
print("...")

# Scegli i moduli target in base a quelli disponibili
# Questi sono moduli comuni nei modelli LLaMA-based
target_modules = []
for candidate in ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"]:
    if any(candidate in module for module in model_modules):
        target_modules.append(candidate)

if not target_modules:
    # Fallback per altri tipi di modelli
    target_modules = ["query", "key", "value", "dense"]

print(f"Moduli target per LoRA: {target_modules}")

# Configurazione LoRA con rango aumentato
lora_config = LoraConfig(
    r=8,  # Aumentato da 4 a 8 per maggiore capacità di apprendimento
    lora_alpha=32,  # Aumentato proporzionalmente al rango
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.TOKEN_CLS
)


# 3. Ottimizzazioni per il caricamento dei dati
# Utilizzare un DataLoader specializzato che pre-elabora i dati in CPU
from torch.utils.data import DataLoader

def create_optimized_dataloader(dataset, batch_size, shuffle=True):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=2,         # Usa 2 worker per caricare i dati in parallelo
        pin_memory=True,       # Pin memory per trasferimenti CPU->GPU più veloci
        prefetch_factor=2,     # Prefetch per ridurre l'attesa del caricamento dati
    )

# 4. Ottimizza il sampling del dataset
# Se il dataset è molto grande, considera l'uso di un subset
from datasets import Dataset

def sample_dataset(dataset, max_samples=10000):
    if len(dataset) > max_samples:
        indices = np.random.choice(len(dataset), max_samples, replace=False)
        return Dataset.from_dict(dataset[indices])
    return dataset

# 5. Monitoraggio dell'utilizzo di memoria durante l'addestramento
class MemoryMonitorCallback(TrainerCallback):
    def __init__(self, print_every=100):
        self.print_every = print_every

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.print_every == 0:
            print(f"Step {state.global_step}")
            print(f"  Memoria allocata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
            print(f"  Memoria riservata: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
            print(f"  Memoria cache: {torch.cuda.memory_cached() / 1e9:.2f} GB")

# 6. Script per pulire la memoria prima dell'addestramento
def clean_memory():
    # Rilascia memoria Python
    gc.collect()

    # Rilascia memoria PyTorch
    torch.cuda.empty_cache()

    # Forza garbage collection
    import sys
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.device.type == 'cuda':
                print(f"Trovato tensor CUDA non referenziato: {obj.shape}")
                del obj
        except:
            pass

    gc.collect()
    torch.cuda.empty_cache()

    # Stampa memoria disponibile
    print(f"Memoria GPU disponibile: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Memoria GPU allocata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Memoria GPU riservata: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

# Esegui pulizia completa prima dell'addestramento
clean_memory()

# 7. Se le ottimizzazioni non sono sufficienti, considera DeepSpeed
# Installazione: !pip install deepspeed
# E cambia la configurazione di training:

"""
from transformers import TrainingArguments

# Configurazione DeepSpeed ZeRO-2
deepspeed_config = {
    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
        "allgather_partitions": True,
        "allgather_bucket_size": 5e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 5e8,
        "contiguous_gradients": True
    },
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "fp16": {
        "enabled": True,
        "auto_cast": True
    }
}

training_args = TrainingArguments(
    # Altri parametri...
    deepspeed=deepspeed_config,
)
"""

# Applica LoRA al modello
model = get_peft_model(model, lora_config)
print("PEFT/LoRA configurato con rango 8 per un migliore apprendimento")
print(f"Memoria GPU dopo setup LoRA: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Modifiche alla configurazione dell'addestramento per migliorare le performance
training_args = TrainingArguments(
    output_dir="./results_improved",
    eval_strategy="steps",           # Cambiato da "epoch" a "steps"
    eval_steps=100,                  # Valuta ogni 100 steps invece che ogni epoca
    save_strategy="steps",           # Cambiato da "epoch" a "steps"
    save_steps=100,                  # Salva ogni 100 steps invece che ogni epoca
    save_total_limit=3,              # Mantieni solo gli ultimi 3 checkpoint
    learning_rate=5e-5,
    per_device_train_batch_size=2,   # Aumentato da 1 a 2
    per_device_eval_batch_size=4,    # Aumentato da 1 a 4
    num_train_epochs=5,              # Ridotto da 15 a 5
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    gradient_accumulation_steps=16,  # Ridotto da 64 a 16
    dataloader_pin_memory=True,      # Cambiato a True per migliorare il trasferimento dati
    optim="adamw_8bit",
    fp16=True,
    torch_compile=True,              # Abilitato torch_compile per PyTorch 2.0+
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    no_cuda=False,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
)

# Crea un data collator specializzato per NER
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Definizione di un Trainer personalizzato estendendo la classe Trainer
class CustomTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.args.device) if class_weights is not None else None

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None): # Include num_items_in_batch as an argument here
        labels = inputs.pop("labels")
        outputs = model(**inputs)

        # Implementazione del class weighting
        if self.class_weights is not None:
            logits = outputs.logits
            active_loss = labels.view(-1) != -100
            active_logits = logits.view(-1, logits.shape[-1])[active_loss]
            active_labels = labels.view(-1)[active_loss]
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
            loss = loss_fct(active_logits, active_labels)
        else:
            # Default loss
            if isinstance(outputs, dict) and "loss" in outputs:
                loss = outputs["loss"]
            else:
                # Perdita standard per classificazione token
                loss_fct = torch.nn.CrossEntropyLoss()
                logits = outputs.logits
                active_loss = labels.view(-1) != -100
                active_logits = logits.view(-1, logits.shape[-1])[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)

        return (loss, outputs) if return_outputs else loss

# Aggiunta della callback al trainer
trainer = CustomTrainer(
    class_weights=class_weights_tensor,
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    callbacks=[TimingCallback()],
)

# Prima dell'addestramento, verifica se torch.compile è disponibile
import torch
if hasattr(torch, "_dynamo") and torch.__version__ >= "2.0.0":
    print("torch.compile è disponibile e verrà utilizzato")
else:
    print("torch.compile non è disponibile, disabilitazione...")
    training_args.torch_compile = False
# Libera ancora memoria prima dell'addestramento
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU prima dell'addestramento: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Avvio dell'addestramento
print("Avvio addestramento con 15 epoche...")
trainer.train()

# Salvataggio del modello addestrato
print("Salvataggio del modello addestrato...")
output_dir = "./model_ner_improved"
trainer.save_model(output_dir)
print(f"Modello migliorato salvato in: {output_dir}")

# Valutazione sul set di test
print("\nValutazione sul set di test...")
predictions = trainer.predict(tokenized_datasets["test"])

# Estrai predizioni e etichette reali
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

# Converti le predizioni e le etichette in formato leggibile
# Funzione per convertire gli ID back alle etichette originali e rimuovere i padding e i token speciali (-100)
def decode_predictions(preds, labels, id2label):
    decoded_preds = []
    decoded_labels = []

    for pred, label in zip(preds, labels):
        pred_tags = []
        label_tags = []

        for p, l in zip(pred, label):
            if l != -100:  # Ignora i token padding/speciali
                pred_tags.append(id2label[p])
                label_tags.append(id2label[l])

        decoded_preds.append(pred_tags)
        decoded_labels.append(label_tags)

    return decoded_preds, decoded_labels

# PRIMA chiama la funzione decode_predictions
decoded_preds, decoded_labels = decode_predictions(preds, labels, id2label)

# POI usa le variabili restituite nelle stampe
print(f"# decoded_preds: {len(decoded_preds)} esempi")
print(f"# decoded_labels: {len(decoded_labels)} esempi")
print(f"Esempio predizione: {decoded_preds[0] if decoded_preds else 'Nessuna'}")
print(f"Esempio etichetta: {decoded_labels[0] if decoded_labels else 'Nessuna'}")

# Verifica che non siano solo 'O' (nessuna entità)
flatten = lambda l: [item for sublist in l for item in sublist]
all_labels = set(flatten(decoded_labels))
all_preds = set(flatten(decoded_preds))

print(f"Etichette vere uniche: {all_labels}")
print(f"Etichette predette uniche: {all_preds}")

# Calcola le metriche di valutazione
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# Installa seqeval se non è disponibile
try:
    import seqeval
except ImportError:
    !pip install seqeval -q
    from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# Calcola e stampa le metriche principali
print("\nMetriche di valutazione:")
print(f"Precision: {precision_score(decoded_labels, decoded_preds):.4f}")
print(f"Recall: {recall_score(decoded_labels, decoded_preds):.4f}")
print(f"F1 Score: {f1_score(decoded_labels, decoded_preds):.4f}")

# Stampa il report di classificazione completo
print("\nReport di classificazione dettagliato:")
print(classification_report(decoded_labels, decoded_preds))

Memoria GPU disponibile: 15.83 GB
Memoria GPU utilizzata: 0.00 GB
Etichette caricate: ['O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-LAW', 'I-LAW', 'B-ACT', 'I-ACT', 'B-ORG', 'I-ORG', 'B-OPA', 'I-OPA']
Mapping etichette -> ID: {'O': 0, 'B-LOC': 1, 'I-LOC': 2, 'B-PER': 3, 'I-PER': 4, 'B-LAW': 5, 'I-LAW': 6, 'B-ACT': 7, 'I-ACT': 8, 'B-ORG': 9, 'I-ORG': 10, 'B-OPA': 11, 'I-OPA': 12}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Caricamento dei dataset...
Analisi delle lunghezze delle sequenze:
Lunghezza media delle sequenze nel train set: 29.2 token
Sequenza più lunga nel train set: 246 token
Percentile 95 delle lunghezze: 86 token
Percentile 99 delle lunghezze: 182 token
Analisi della distribuzione delle etichette:
Etichette uniche nel dataset: {'I-PER', 'I-LAW', 'I-LOC', 'O', 'I-ORG', 'I-ACT', 'I-OPA'}
  - I-PER: 184 occorrenze
  - I-LAW: 866 occorrenze
  - I-LOC: 298 occorrenze
  - O: 6736 occorrenze
  - I-ORG: 215 occorrenze
  - I-ACT: 246 occorrenze
  - I-OPA: 165 occorrenze
Pesi calcolati per le classi: {'I-PER': 6.762422360248447, 'I-LAW': 1.4368195315077532, 'I-LOC': 4.175455417066155, 'O': 0.18472175093315235, 'I-ORG': 5.787375415282392, 'I-ACT': 5.0580720092915215, 'I-OPA': 7.541125541125541}
Tensor pesi classi: tensor([0.1847, 1.0000, 4.1755, 1.0000, 6.7624, 1.0000, 1.4368, 1.0000, 5.0581,
        1.0000, 5.7874, 1.0000, 7.5411])
Pre-processing dei dataset...


Map:   0%|          | 0/298 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

...

torch.compile è disponibile e verrà utilizzato
Memoria GPU prima dell'addestramento: 5.79 GB
Avvio addestramento con 15 epoche...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


Step 10: 0.04 steps/s, 22.336 s/step
Epoca 1.0 completata. Tempo totale: 3.72 minuti
Step 20: 0.05 steps/s, 20.608 s/step
Epoca 2.0 completata. Tempo totale: 7.16 minuti
Step 30: 0.05 steps/s, 20.578 s/step
Epoca 3.0 completata. Tempo totale: 10.59 minuti
Step 40: 0.05 steps/s, 20.578 s/step
Epoca 4.0 completata. Tempo totale: 14.02 minuti
Epoca 4.5369127516778525 completata. Tempo totale: 15.89 minuti
Salvataggio del modello addestrato...
Modello migliorato salvato in: ./model_ner_improved

Valutazione sul set di test...


# decoded_preds: 36 esempi
# decoded_labels: 36 esempi
Esempio predizione: ['I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LAW', 'I-ORG', 'I-PER', 'O']
Esempio etichetta: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'I-PER', 'O']
Etichette vere uniche: {'I-PER', 'I-LAW', 'I-LOC', 'O', 'I-ORG', 'I-ACT', 'I-OPA'}
Etichette predette uniche: {'I-PER', 'I-LAW', 'I-LOC', 'O', 'I-ORG', 'I-ACT', 'I-OPA'}

Metriche di valutazione:
Precision: 0.0408
Recall: 0.2222
F1 Score: 0.0690

Report di classificazione dettagliato:
              precision    recall  f1-score   support

         ACT       0.04      0.25      0.06         4
         LAW       0.18      0.50      0.26         6
         LOC       0.00      0.00      0.00         5
         OPA       0.11      0.50      0.18         2
         ORG       0.05      0.50      0.10         2
         PER       0.00      0.00      0.00         8

   micro avg       0.04      0.22      0.07       

salvataggio del modello addestrato

In [ ]:
# Salvataggio del modello addestrato
!pip install seqeval -q
print("Salvataggio del modello addestrato...")
output_dir = "/content/model_ner_saved"
trainer.save_model(output_dir)
print(f"Modello migliorato salvato in: {output_dir}")

# Ottieni predizioni per il set di test
print("\nValutazione sul set di test...")
predictions = trainer.predict(tokenized_datasets["test"])

# Estrai predizioni e etichette reali
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

# Converti le predizioni e le etichette in formato leggibile
decoded_preds = []
decoded_labels = []

# Funzione per convertire gli ID back alle etichette originali e rimuovere i padding e i token speciali (-100)
def decode_predictions(preds, labels, id2label):
    decoded_preds = []
    decoded_labels = []

    for pred, label in zip(preds, labels):
        pred_tags = []
        label_tags = []

        for p, l in zip(pred, label):
            if l != -100:  # Ignora i token padding/speciali
                pred_tags.append(id2label[p])
                label_tags.append(id2label[l])

        decoded_preds.append(pred_tags)
        decoded_labels.append(label_tags)

    return decoded_preds, decoded_labels

decoded_preds, decoded_labels = decode_predictions(preds, labels, id2label)

# Calcola le metriche di valutazione
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# Installa seqeval se non è disponibile
try:
    import seqeval
except ImportError:
    !pip install seqeval -q
    from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# Calcola e stampa le metriche principali
print("\nMetriche di valutazione:")
print(f"Precision: {precision_score(decoded_labels, decoded_preds):.4f}")
print(f"Recall: {recall_score(decoded_labels, decoded_preds):.4f}")
print(f"F1 Score: {f1_score(decoded_labels, decoded_preds):.4f}")

# Stampa il report di classificazione completo
print("\nReport di classificazione dettagliato:")
print(classification_report(decoded_labels, decoded_preds))

# Analisi delle predizioni su alcuni esempi
print("\nEsempi di predizioni:")
num_examples = min(5, len(decoded_preds))

# Carica i token originali per gli esempi
test_tokens = test_tokens[:num_examples]

for i in range(num_examples):
    tokens = test_tokens[i]
    pred_tags = decoded_preds[i]
    true_tags = decoded_labels[i]

    # Assicuriamoci che le liste abbiano la stessa lunghezza (potrebbe non essere sempre così a causa della tokenizzazione)
    min_len = min(len(tokens), len(pred_tags), len(true_tags))

    print(f"\nEsempio {i+1}:")
    print(f"{'Token':<15} | {'Etichetta vera':<15} | {'Etichetta predetta':<15}")
    print("-" * 50)

    for j in range(min_len):
        print(f"{tokens[j]:<15} | {true_tags[j]:<15} | {pred_tags[j]:<15}")

# Come utilizzare il modello per nuove predizioni
print("\nEsempio di come utilizzare il modello per nuove predizioni:")
print("""
# Carica il modello salvato
from transformers import AutoModelForTokenClassification, AutoTokenizer
from peft import PeftModel, PeftConfig

# Carica la configurazione PEFT
peft_config = PeftConfig.from_pretrained("./model_ner_saved")

# Carica il modello base
base_model = AutoModelForTokenClassification.from_pretrained(
    peft_config.base_model_name_or_path,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    device_map="auto"
)

# Carica il modello PEFT
model = PeftModel.from_pretrained(base_model, "./model_ner_saved")
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

# Funzione per predire entità in un nuovo testo
def predict_entities(text):
    # Tokenizza il testo
    tokens = text.split()
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")

    # Sposta gli input sulla GPU se disponibile
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # Ottieni le predizioni
    with torch.no_grad():
        outputs = model(**inputs)

    # Converti le predizioni in etichette
    predictions = outputs.logits.argmax(dim=-1)[0].cpu().numpy()

    # Converti gli ID delle predizioni in etichette
    word_ids = inputs.word_ids()[0]

    result = []
    previous_word_idx = None

    for idx, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx == previous_word_idx:
            continue

        pred_id = predictions[idx]
        pred_label = id2label[pred_id]
        result.append((tokens[word_idx], pred_label))

        previous_word_idx = word_idx

    return result

# Esempio di uso
example_text = "IL SINDACO MARIO ROSSI ha approvato la delibera del COMUNE DI MILANO"
entities = predict_entities(example_text)
print("Entità rilevate:")
for token, entity in entities:
    print(f"{token}: {entity}")
""")

Salvataggio del modello addestrato...
Modello migliorato salvato in: /content/model_ner_saved

Valutazione sul set di test...



Metriche di valutazione:
Precision: 0.0408
Recall: 0.2222
F1 Score: 0.0690

Report di classificazione dettagliato:
              precision    recall  f1-score   support

         ACT       0.04      0.25      0.06         4
         LAW       0.18      0.50      0.26         6
         LOC       0.00      0.00      0.00         5
         OPA       0.11      0.50      0.18         2
         ORG       0.05      0.50      0.10         2
         PER       0.00      0.00      0.00         8

   micro avg       0.04      0.22      0.07        27
   macro avg       0.06      0.29      0.10        27
weighted avg       0.06      0.22      0.09        27


Esempi di predizioni:

Esempio 1:
Token           | Etichetta vera  | Etichetta predetta
--------------------------------------------------
Di              | O               | I-PER          
trasmettere     | O               | I-PER          
,               | O               | O              
per             | O               | O       

In [ ]:
def process_ner_data(test_file="test_sampled_iob_corretto.txt", classes_file="classes.txt"):
    print("Inizio elaborazione dei file NER...")
    print(f"- File di test: {test_file}")
    print(f"- File delle classi: {classes_file}")

    # Verifica esistenza file
    import os
    if not os.path.exists(test_file):
        print(f"ERRORE: File test {test_file} non trovato")
        print(f"Directory corrente: {os.getcwd()}")
        print(f"Contenuto directory: {os.listdir('.')}")
        return

    if not os.path.exists(classes_file):
        print(f"ERRORE: File classi {classes_file} non trovato")
        return

    # Leggi e mostra il contenuto parziale dei file per verifica
    print("\nContenuto parziale del file di test:")
    with open(test_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()[:10]  # Prime 10 righe
        for line in lines:
            print(line.strip())

    print("\nContenuto del file delle classi:")
    with open(classes_file, 'r', encoding='utf-8') as f:
        for line in f:
            print(line.strip())

    print("\nElaborazione completata con successo!")

# Chiamata diretta alla funzione
process_ner_data()

Inizio elaborazione dei file NER...
- File di test: test_sampled_iob_corretto.txt
- File delle classi: classes.txt

Contenuto parziale del file di test:
Di	O
trasmettere	O
,	O
per	O
opportuna	O
conoscenza	O
,	O
copia	O
del	O
presente	O

Contenuto del file delle classi:
O
B-LOC
I-LOC
B-PER
I-PER
B-LAW
I-LAW
B-ACT
I-ACT
B-ORG
I-ORG
B-OPA
I-OPA

Elaborazione completata con successo!


inferenza

In [ ]:
# Soluzione 1: Inferenza con modello già addestrato
# Da usare se avete il modello già addestrato dal codice originale

import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

    # Carica il modello già addestrato
def inference_with_trained_model():
    # Pulizia della memoria prima di caricare il modello
    import gc
    import torch
    import os
    from peft import PeftModel, PeftConfig

    print("Pulizia memoria prima di caricare il modello...")
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Carica il modello già addestrato
    model_path = "/content/model_ner_improved"  # Percorso al modello salvato

    try:
        # Verifica se è un modello PEFT/LoRA
        is_peft_model = os.path.exists(os.path.join(model_path, "adapter_config.json"))

        if is_peft_model:
            print("Rilevato modello PEFT/LoRA. Caricamento del modello base e adattatore...")

            # Usa un approccio alternativo: carica prima la configurazione PEFT
            peft_config = PeftConfig.from_pretrained(model_path)
            base_model_name = peft_config.base_model_name_or_path

            # Se la base_model_name_or_path è vuota, usa il modello di default
            if not base_model_name:
                base_model_name = "expertai/LLaMAntino-3-SLIMER-IT"
                print(f"Nome del modello base non trovato in adapter_config.json, uso il default: {base_model_name}")

            # Carica il tokenizer dal modello base
            from transformers import AutoTokenizer
            tokenizer = AutoTokenizer.from_pretrained(base_model_name)

            # Carica le informazioni sulle etichette dal file config.json
            import json
            config_path = os.path.join(model_path, "config.json")
            if os.path.exists(config_path):
                with open(config_path, 'r') as f:
                    config_data = json.load(f)
                    num_labels = config_data.get("num_labels", 13)
                    id2label = config_data.get("id2label", {})
                    label2id = config_data.get("label2id", {})
            else:
                # Se non c'è il config.json, carica le etichette dal file classes.txt
                with open("classes.txt", "r", encoding="utf-8") as f:
                    label_list = [line.strip() for line in f.readlines()]
                num_labels = len(label_list)
                id2label = {str(i): l for i, l in enumerate(label_list)}
                label2id = {l: str(i) for i, l in enumerate(label_list)}

            # Prova un approccio alternativo: carica tutto insieme
            from transformers import AutoModelForTokenClassification

            # Importante: senza device_map il modello userà la CPU
            # È più lento ma evita problemi di memoria GPU
            print("Caricamento del modello in CPU per evitare problemi di memoria...")
            base_model = AutoModelForTokenClassification.from_pretrained(
                base_model_name,
                num_labels=num_labels,
                id2label=id2label,
                label2id=label2id,
                torch_dtype=torch.float16,
                device_map="cpu"  # Forza il caricamento sulla CPU
            )

            # Ora carica l'adattatore PEFT (anche sulla CPU)
            model = PeftModel.from_pretrained(base_model, model_path, device_map="cpu")

            # Se c'è una GPU disponibile, sposta solo le parti che ci stanno
            if torch.cuda.is_available():
                try:
                    print("Tentativo di spostare alcune parti del modello su GPU...")
                    # Sposta solo alcune parti essenziali su GPU
                    model.base_model.model.embed_tokens = model.base_model.model.embed_tokens.to("cuda:0")
                    model.base_model.lm_head = model.base_model.lm_head.to("cuda:0")
                    # Potresti aggiungere altri componenti ma stai attento alla memoria
                except Exception as e:
                    print(f"Non è stato possibile spostare componenti su GPU: {e}")
                    print("Il modello rimarrà sulla CPU (sarà più lento ma funzionerà)")
        else:
            # Codice per modello non-PEFT rimane invariato
            print("Tentativo di caricamento come modello standard...")
            from transformers import AutoModelForTokenClassification, AutoTokenizer
            model = AutoModelForTokenClassification.from_pretrained(
                model_path,
                local_files_only=True,
                device_map="cpu",  # Usa la CPU per sicurezza
                torch_dtype=torch.float16
            )
            tokenizer = AutoTokenizer.from_pretrained("expertai/LLaMAntino-3-SLIMER-IT")

        # [resto del codice rimane invariato]

        model.eval()  # Imposta il modello in modalità valutazione
        print("Modello caricato con successo!")

        # Funzione per fare predizioni su un testo
        def predict_entities(text):
            # Dividi il testo in batch più piccoli (es. 50 token per batch)
            tokens = text.split()
            batch_size = 50
            batches = [tokens[i:i+batch_size] for i in range(0, len(tokens), batch_size)]

            all_predicted_labels = []

            for batch in batches:
                # Tokenizza il batch
                inputs = tokenizer(batch, is_split_into_words=True, return_tensors="pt")

                # Sposta gli input sulla stessa periferica del modello
                if hasattr(model, "device"):
                    device = model.device
                    inputs = {k: v.to(device) for k, v in inputs.items()}

                # Predizioni con maggiore efficienza di memoria
                with torch.no_grad():
                    with torch.cuda.amp.autocast(enabled=True):  # Usa precisione mista
                        outputs = model(**inputs)

                # Ottieni le etichette predette per questo batch
                predictions = outputs.logits.argmax(-1).squeeze().cpu().tolist()  # Assicurati che siano su CPU

                # Gestisci il caso in cui predictions sia un singolo numero (batch con un solo token)
                if not isinstance(predictions, list):
                    predictions = [predictions]

                # Converti le predizioni in etichette
                word_ids = inputs.word_ids(batch_index=0)

                current_word = None
                current_label = None

                # Aggiungi solo le entità rilevanti
                for idx, word_id in enumerate(word_ids):
                    if word_id is None or idx >= len(predictions):
                        continue

                    if word_id != current_word:
                        current_word = word_id
                        label_id = predictions[idx]
                        # Converti ID etichetta in testo
                        if label_id in model.config.id2label:
                            current_label = model.config.id2label[label_id]
                            if current_label != "O":  # Ignora l'etichetta "O" (nessuna entità)
                                all_predicted_labels.append((batch[word_id], current_label))

            # Libera memoria
            torch.cuda.empty_cache()
            gc.collect()

            return all_predicted_labels

        # Esempio di utilizzo
        sample_text = "La conferenza si terrà a Milano il prossimo mese"
        entities = predict_entities(sample_text)

        print(f"Testo: {sample_text}")
        print("Entità rilevate:")
        for entity, label in entities:
            print(f"  - {entity}: {label}")

        return True

    except RuntimeError as e:
            if "CUDA out of memory" in str(e) or "alloc" in str(e).lower():
                print("Errore di memoria durante l'inferenza. Prova queste soluzioni:")
                print("1. Ridurre la dimensione del batch (batch_size)")
                print("2. Utilizzare un ambiente con più memoria GPU/RAM")
                print("3. Utilizzare un modello più piccolo")
            else:
                print(f"Errore nel caricamento del modello: {e}")
                print("Il modello addestrato non è disponibile. Prova un'altra soluzione.")
            return False

# Soluzione 2: Few-Shot Learning
# Divide il test set in train/val/test e addestra un nuovo modello

def few_shot_learning(test_file, classes_file):
    import torch
    import numpy as np
    from transformers import (
        AutoModelForTokenClassification,
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        DataCollatorForTokenClassification
    )
    from datasets import Dataset
    from sklearn.model_selection import train_test_split
    from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

    print("Inizializzazione del Few-Shot Learning...")

    # Carica il file delle classi
    with open(classes_file, "r", encoding="utf-8") as f:
        class_list = [line.strip() for line in f.readlines()]
        # Non duplicare "O" se è già presente nel file
    if "O" not in class_list:
        class_list = ["O"] + class_list
    label2id = {l: i for i, l in enumerate(class_list)}
    id2label = {i: l for l, i in label2id.items()}

    print(f"Caricate {len(class_list)} classi")
    # Verifica che il numero di etichette corrisponda
    num_labels = len(class_list)
    print(f"Caricate {num_labels} classi: {class_list}")
    print(f"Mappatura ID-etichetta: {id2label}")

    # Funzione per caricare dati IOB
    def load_iob_dataset(path):
        sentences, labels = [], []
        with open(path, encoding='utf-8') as f:
            tokens, tags = [], []
            for line in f:
                line = line.strip()
                if not line:
                    if tokens:
                        sentences.append(tokens)
                        labels.append(tags)
                        tokens, tags = [], []
                else:
                    splits = line.split()
                    if len(splits) >= 2:
                        tokens.append(splits[0])
                        tags.append(splits[-1])
            if tokens:
                sentences.append(tokens)
                labels.append(tags)
        return sentences, labels

    # Carica il test set
    tokens, tags = load_iob_dataset(test_file)
    print(f"Caricati {len(tokens)} esempi dal file di test")

    # Dividi il test set in train/val/test
    train_tokens, temp_tokens, train_tags, temp_tags = train_test_split(
        tokens, tags, test_size=0.2, random_state=42
    )
    val_tokens, test_tokens, val_tags, test_tags = train_test_split(
        temp_tokens, temp_tags, test_size=0.5, random_state=42
    )

    print(f"Split del dataset: Train={len(train_tokens)}, Val={len(val_tokens)}, Test={len(test_tokens)}")

    # Carica il tokenizer
    model_name = "expertai/LLaMAntino-3-SLIMER-IT"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Funzione per tokenizzare e allineare le etichette
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["tokens"],
            is_split_into_words=True,
            truncation=True,
            padding="max_length",
            max_length=32,
            return_tensors=None
        )

        labels = []
        for i, label in enumerate(examples["ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []

            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_text = label[word_idx]
                    label_id = label2id.get(label_text, 0)
                    label_ids.append(label_id)
                else:
                    label_ids.append(-100)

                previous_word_idx = word_idx

            labels.append(label_ids)

        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    # Crea i dataset
    train_ds = Dataset.from_dict({"tokens": train_tokens, "ner_tags": train_tags})
    val_ds = Dataset.from_dict({"tokens": val_tokens, "ner_tags": val_tags})
    test_ds = Dataset.from_dict({"tokens": test_tokens, "ner_tags": test_tags})

    # Tokenizza i dataset
    tokenized_train = train_ds.map(tokenize_and_align_labels, batched=True)
    tokenized_val = val_ds.map(tokenize_and_align_labels, batched=True)
    tokenized_test = test_ds.map(tokenize_and_align_labels, batched=True)

    print("Dataset tokenizzati con successo")

    # Carica il modello base e configura LoRA per training efficiente
    print("Configurazione del modello per few-shot learning con LoRA...")

    # Configurazione base con meno parametri per un training più veloce
    from transformers import BitsAndBytesConfig

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=num_labels,  # Questo deve corrispondere alla lunghezza di id2label
        id2label=id2label,
        label2id=label2id,
        device_map="auto",
        quantization_config=quantization_config,
        torch_dtype=torch.float16
    )

    # Preparazione per LoRA
    model = prepare_model_for_kbit_training(model)

    # Configurazione LoRA semplificata
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

    lora_config = LoraConfig(
        r=4,
        lora_alpha=16,
        target_modules=target_modules,
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.TOKEN_CLS
    )

    model = get_peft_model(model, lora_config)
    print("Modello configurato con LoRA")

    # Configurazione training veloce
    training_args = TrainingArguments(
        output_dir="./results_few_shot",
        learning_rate=5e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=4,
        num_train_epochs=3,  # Ridotto per few-shot
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=50,
        gradient_accumulation_steps=8,
        fp16=True,
        gradient_checkpointing=True,
        report_to="none",  # Imposta esplicitamente "none" per disabilitare wandb
    )


    # Collator e trainer
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        data_collator=data_collator,
    )

    print("Inizio addestramento few-shot...")
    trainer.train()

    # Salva il modello
    trainer.save_model("./model_few_shot")
    print("Modello few-shot salvato in ./model_few_shot")

    # Valutazione sul set di test
    print("Valutazione sul set di test...")
    results = trainer.evaluate(tokenized_test)
    print(f"Risultati valutazione: {results}")

    return True

# Soluzione 3: Transfer Learning da modello NER pre-addestrato
# Usa un modello già addestrato per NER in italiano

def transfer_learning_ner(test_file, classes_file):
    import torch
    from transformers import (
        AutoModelForTokenClassification,
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        DataCollatorForTokenClassification
    )
    from datasets import Dataset
    from sklearn.model_selection import train_test_split

    print("Inizializzazione del Transfer Learning da modello NER pre-addestrato...")

    # Carica il file delle classi
    with open(classes_file, "r", encoding="utf-8") as f:
        target_classes = [line.strip() for line in f.readlines()]

    # Non duplicare "O" se è già presente nel file
    if "O" not in target_classes:
        target_classes = ["O"] + target_classes

    target_label2id = {l: i for i, l in enumerate(target_classes)}
    target_id2label = {i: l for i, l in target_label2id.items()}

    # Usa un modello pre-addestrato esistente su Hugging Face
    # Il modello precedente sembra errato, quindi sostituiscilo con questi modelli verificati:
    # Opzioni corrette per modelli NER in italiano:
    # - "dbmdz/bert-base-italian-cased" (modello base italiano)
    # - "dbmdz/bert-base-italian-xxl-cased" (versione più grande)
    # - "Musixmatch/umberto-commoncrawl-cased-v1" (altro modello italiano)
    pretrained_model_name = "dbmdz/bert-base-italian-cased"

    print(f"Caricamento del modello pre-addestrato: {pretrained_model_name}")
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

    # Carica il modello pre-addestrato
    model = AutoModelForTokenClassification.from_pretrained(
        pretrained_model_name,
        num_labels=len(target_classes),
        id2label=target_id2label,
        label2id=target_label2id
    )

    print(f"Modello pre-addestrato {pretrained_model_name} caricato")

    # Mostra le classi originali del modello pre-addestrato
    original_labels = model.config.id2label if hasattr(model.config, "id2label") else {}
    print(f"Etichette originali del modello: {list(original_labels.values())}")
    print(f"Etichette target: {target_classes}")

    # [Qui dovresti aggiungere il codice per l'addestramento]
    # Questo codice dovrebbe essere simile a quello nella funzione few_shot_learning()
    print("Il transfer learning è impostato e pronto per l'addestramento.")

    # Per rendere questa funzione più completa, potresti voler aggiungere il codice effettivo
    # di addestramento, simile a quello che hai nella funzione few_shot_learning()
    return True

# Soluzione 4: Analisi del test set
# Utile per comprendere il dataset quando non si può addestrare un modello

def analyze_test_set(test_file, classes_file):
    import matplotlib.pyplot as plt
    import pandas as pd
    from collections import Counter

    print("Analisi del test set...")

    # Carica le classi
    with open(classes_file, "r", encoding="utf-8") as f:
        classes = [line.strip() for line in f.readlines()]
    classes = ["O"] + classes

    # Funzione per caricare dati IOB
    def load_iob_dataset(path):
        sentences, labels = [], []
        with open(path, encoding='utf-8') as f:
            tokens, tags = [], []
            for line in f:
                line = line.strip()
                if not line:
                    if tokens:
                        sentences.append(tokens)
                        labels.append(tags)
                        tokens, tags = [], []
                else:
                    splits = line.split()
                    if len(splits) >= 2:
                        tokens.append(splits[0])
                        tags.append(splits[-1])
            if tokens:
                sentences.append(tokens)
                labels.append(tags)
        return sentences, labels

    # Carica il test set
    tokens, tags = load_iob_dataset(test_file)

    # Statistiche base
    num_sequences = len(tokens)
    lengths = [len(t) for t in tokens]
    avg_length = sum(lengths) / len(lengths)
    max_length = max(lengths)
    min_length = min(lengths)

    # Analisi delle etichette
    all_tags = [tag for seq in tags for tag in seq]
    tag_counts = Counter(all_tags)

    # Ordinamento delle etichette per frequenza
    sorted_tags = sorted(tag_counts.items(), key=lambda x: x[1], reverse=True)

    # Output delle statistiche
    print(f"Numero di sequenze: {num_sequences}")
    print(f"Lunghezza media: {avg_length:.1f} token")
    print(f"Lunghezza massima: {max_length} token")
    print(f"Lunghezza minima: {min_length} token")
    print("\nDistribuzione delle etichette:")
    for tag, count in sorted_tags:
        print(f"{tag}: {count} ({count/len(all_tags)*100:.1f}%)")

    # Crea un DataFrame per le etichette
    tag_df = pd.DataFrame(sorted_tags, columns=["Tag", "Count"])
    tag_df["Percentage"] = tag_df["Count"] / len(all_tags) * 100

    print("\nAnalisi delle sequenze per tipologia di entità:")
    # Conta quante sequenze contengono almeno un'istanza di ciascuna etichetta
    entity_in_sequences = {}
    for i, seq_tags in enumerate(tags):
        for tag in set(seq_tags):
            if tag not in entity_in_sequences:
                entity_in_sequences[tag] = 0
            entity_in_sequences[tag] += 1

    for tag, count in sorted(entity_in_sequences.items(), key=lambda x: x[1], reverse=True):
        if tag != "O":  # Escludi l'etichetta O
            print(f"{tag}: presente in {count} sequenze ({count/num_sequences*100:.1f}%)")

    print("\nAnalisi completata con successo!")
    return True

# Funzione principale che prova le diverse soluzioni
# Modifica la funzione process_ner_data() in questo modo:
def process_ner_data(test_file="test_sampled_iob_corretto.txt", classes_file="classes.txt"):
    print(f"Elaborazione dei dati NER usando i file:")
    print(f"- File di test: {test_file}")
    print(f"- File delle classi: {classes_file}")

    # Chiedi all'utente quale soluzione vuole utilizzare
    print("\nScegli la soluzione da utilizzare:")
    print("1. Inferenza con modello già addestrato (richiede molta memoria)")
    print("2. Analisi del test set (basso consumo di memoria)")
    print("3. Few-Shot Learning (addestramento su subset del test)")
    print("4. Transfer Learning (da modello NER pre-addestrato)")

    choice = input("Scelta (1/2/3/4): ")

    if choice == "1":
        print("\nEsecuzione dell'inferenza con modello già addestrato...")
        if inference_with_trained_model():
            print("\nInferenza completata con successo usando il modello addestrato.")
            return
        else:
            print("\nNon è stato possibile eseguire l'inferenza. Prova un'altra soluzione.")

    elif choice == "2":
        print("\nEsecuzione dell'analisi del test set...")
        analyze_test_set(test_file, classes_file)

    elif choice == "3":
        print("\nAvvio del Few-Shot Learning...")
        few_shot_learning(test_file, classes_file)

    elif choice == "4":
        print("\nAvvio del Transfer Learning...")
        transfer_learning_ner(test_file, classes_file)

    else:
        print("Scelta non valida. Elaborazione terminata.")

    print("\nElaborazione completata.")
if __name__ == "__main__":
    try:
        # Specifica i percorsi esatti dei file, modificali in base alla tua struttura
        test_file = "test_sampled_iob_corretto.txt"
        classes_file = "classes.txt"

        # Esegui la funzione principale
        process_ner_data(test_file, classes_file)
    except Exception as e:
        print(f"ERRORE nell'esecuzione: {str(e)}")
        import traceback
        traceback.print_exc()

Elaborazione dei dati NER usando i file:
- File di test: test_sampled_iob_corretto.txt
- File delle classi: classes.txt

Scegli la soluzione da utilizzare:
1. Inferenza con modello già addestrato (richiede molta memoria)
2. Analisi del test set (basso consumo di memoria)
3. Few-Shot Learning (addestramento su subset del test)
4. Transfer Learning (da modello NER pre-addestrato)
Scelta (1/2/3/4): 4

Avvio del Transfer Learning...
Inizializzazione del Transfer Learning da modello NER pre-addestrato...
Caricamento del modello pre-addestrato: dbmdz/bert-base-italian-cased


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modello pre-addestrato dbmdz/bert-base-italian-cased caricato
Etichette originali del modello: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Etichette target: ['O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-LAW', 'I-LAW', 'B-ACT', 'I-ACT', 'B-ORG', 'I-ORG', 'B-OPA', 'I-OPA']
Il transfer learning è impostato e pronto per l'addestramento.

Elaborazione completata.


codice secondo non ottimizzato

In [ ]:
# Librerie necessarie
!pip install bitsandbytes accelerate -q
!pip install -q datasets
!pip install seqeval -q
!pip install deepspeed
import gc
import os
import torch
import numpy as np
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForTokenClassification, BitsAndBytesConfig, DataCollatorForTokenClassification
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training, PeftModel, PeftConfig
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers.integrations import TensorBoardCallback
import time
from transformers import TrainerCallback
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# Configurazione CUDA per evitare problemi di frammentazione memoria
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Per essere sicuri di liberare tutta la memoria disponibile
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU disponibile: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Memoria GPU utilizzata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Callback per monitorare tempi di addestramento
class TimingCallback(TrainerCallback):
    def __init__(self):
        self.start_time = time.time()
        self.step_start_time = time.time()
        self.steps = 0

    def on_step_end(self, args, state, control, **kwargs):
        self.steps += 1
        if self.steps % 10 == 0:  # Stampa ogni 10 steps
            elapsed = time.time() - self.step_start_time
            steps_per_second = 10 / elapsed
            print(f"Step {state.global_step}: {steps_per_second:.2f} steps/s, {elapsed/10:.3f} s/step")
            self.step_start_time = time.time()

    def on_epoch_end(self, args, state, control, **kwargs):
        elapsed = time.time() - self.start_time
        print(f"Epoca {state.epoch} completata. Tempo totale: {elapsed/60:.2f} minuti")

# Callback per monitorare l'utilizzo della memoria
class MemoryMonitorCallback(TrainerCallback):
    def __init__(self, print_every=100):
        self.print_every = print_every

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.print_every == 0:
            print(f"Step {state.global_step}")
            print(f"  Memoria allocata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
            print(f"  Memoria riservata: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
            print(f"  Memoria cache: {torch.cuda.memory_cached() / 1e9:.2f} GB")

# Define the label_list, label2id, and id2label
with open("classes.txt", "r", encoding="utf-8") as f:
    label_list = [line.strip() for line in f.readlines()]
label_list = ["O"] + label_list  # aggiungiamo la classe 'O'
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

print(f"Etichette caricate: {label_list}")
print(f"Mapping etichette -> ID: {label2id}")

# Define the load_iob_dataset function
def load_iob_dataset(path):
    sentences, labels = [], []
    with open(path, encoding='utf-8') as f:
        tokens, tags = [], []
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append(tokens)
                    labels.append(tags)
                    tokens, tags = [], []
            else:
                splits = line.split()
                if len(splits) >= 2:  # Assicurati che ci siano almeno due colonne
                    tokens.append(splits[0])
                    tags.append(splits[-1])
        if tokens:
            sentences.append(tokens)
            labels.append(tags)
    return sentences, labels

# Carica il tokenizer
model_name = "expertai/LLaMAntino-3-SLIMER-IT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Funzione per tokenizzare e allineare le etichette
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=32,              # Ridotto da 64 a 32
        return_tensors=None
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignora token speciali
            elif word_idx != previous_word_idx:
                # Solo il primo token di una parola ottiene l'etichetta
                label_text = label[word_idx]
                label_id = label2id.get(label_text, 0)  # Default a 'O' (0) se l'etichetta non esiste
                label_ids.append(label_id)
            else:
                # Token successivi della stessa parola
                label_ids.append(-100)  # Usiamo -100 per ignorare questi token nella loss

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Carica i dataset
print("Caricamento dei dataset...")
train_tokens, train_tags = load_iob_dataset("/content/train_sampled_iob_corretto.txt")
dev_tokens, dev_tags = load_iob_dataset("/content/dev_sampled_iob_corretto.txt")
test_tokens, test_tags = load_iob_dataset("/content/test_sampled_iob_corretto.txt")

# Analisi della lunghezza delle sequenze nel dataset
print("Analisi delle lunghezze delle sequenze:")
train_lengths = [len(seq) for seq in train_tokens]
print(f"Lunghezza media delle sequenze nel train set: {sum(train_lengths)/len(train_lengths):.1f} token")
print(f"Sequenza più lunga nel train set: {max(train_lengths)} token")
print(f"Percentile 95 delle lunghezze: {sorted(train_lengths)[int(len(train_lengths)*0.95)]} token")
print(f"Percentile 99 delle lunghezze: {sorted(train_lengths)[int(len(train_lengths)*0.99)]} token")

# Verifica la distribuzione delle etichette
print("Analisi della distribuzione delle etichette:")
all_tags = [tag for tag_list in train_tags for tag in tag_list]
unique_tags = set(all_tags)
print(f"Etichette uniche nel dataset: {unique_tags}")
tag_counts = {}
for tag in unique_tags:
    tag_counts[tag] = all_tags.count(tag)
    print(f"  - {tag}: {tag_counts[tag]} occorrenze")

# Calcolo pesi delle classi per class weighting
tag_weights = {}
total_tags = len(all_tags)
for tag in unique_tags:
    # Metodo inverso della frequenza
    tag_weights[tag] = total_tags / (tag_counts[tag] * len(unique_tags))
print(f"Pesi calcolati per le classi: {tag_weights}")

# Converti i pesi in un array per il modello
class_weights = []
for i in range(len(label_list)):
    label = id2label[i]
    if label in tag_weights:
        class_weights.append(tag_weights[label])
    else:
        # Gestisci etichette che potrebbero non essere presenti nei dati di training
        class_weights.append(1.0)

class_weights_tensor = torch.tensor(class_weights)
print(f"Tensor pesi classi: {class_weights_tensor}")

# Crea i dataset
train_ds = Dataset.from_dict({"tokens": train_tokens, "ner_tags": train_tags})
dev_ds = Dataset.from_dict({"tokens": dev_tokens, "ner_tags": dev_tags})
test_ds = Dataset.from_dict({"tokens": test_tokens, "ner_tags": test_tags})

dataset = DatasetDict({
    "train": train_ds,
    "validation": dev_ds,
    "test": test_ds
})

# Pre-processa i dataset
print("Pre-processing dei dataset...")
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["tokens", "ner_tags"]
)
print(f"Dataset processato: {len(tokenized_datasets['train'])} esempi nel train set")

# Verifica la struttura dei dati
print("Verifica della struttura dei dati:")
sample = tokenized_datasets["train"][0]
for key, value in sample.items():
    if isinstance(value, list):
        print(f"{key}: lista di lunghezza {len(value)}, tipo elementi: {type(value[0]).__name__}")
        if len(value) > 0 and isinstance(value[0], (int, float)):
            print(f"  - Esempio valori: {value[:5]}...")
    else:
        print(f"{key}: {type(value).__name__}")

# Funzione per pulire la memoria
def clean_memory():
    # Rilascia memoria Python
    gc.collect()

    # Rilascia memoria PyTorch
    torch.cuda.empty_cache()

    # Forza garbage collection
    import sys
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.device.type == 'cuda':
                print(f"Trovato tensor CUDA non referenziato: {obj.shape}")
                del obj
        except:
            pass

    gc.collect()
    torch.cuda.empty_cache()

    # Stampa memoria disponibile
    print(f"Memoria GPU disponibile: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Memoria GPU allocata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Memoria GPU riservata: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

# Esegui pulizia completa prima dell'addestramento
clean_memory()

# Configura la quantizzazione 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

print("Caricamento del modello con quantizzazione a 4-bit...")
# Crea il modello con quantizzazione a 4-bit e offload
"""
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    device_map="auto",
    quantization_config=quantization_config,
    torch_dtype=torch.float16
)
"""
model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=num_labels,  # Questo deve corrispondere alla lunghezza di id2label
        id2label=id2label,
        label2id=label2id,
        device_map="auto",
        quantization_config=quantization_config,
        torch_dtype=torch.float16
)

# Monitora l'uso di memoria dopo il caricamento del modello
print(f"Memoria GPU dopo caricamento modello: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Prepara il modello per l'addestramento a 4-bit
model = prepare_model_for_kbit_training(model)

# Abilita checkpointing
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()
    print("Gradient checkpointing abilitato")

# Configurazione LoRA con moduli adatti al modello
# Trova quali moduli sono effettivamente nel modello
model_modules = [name for name, _ in model.named_modules()]
print("Moduli disponibili nel modello:")
for module in sorted(set([m.split('.')[-1] for m in model_modules if '.' in m]))[:15]:  # Mostra i nomi dei moduli unici
    print(f"  - {module}")
print("...")

# Scegli i moduli target in base a quelli disponibili
target_modules = []
for candidate in ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"]:
    if any(candidate in module for module in model_modules):
        target_modules.append(candidate)

if not target_modules:
    # Fallback per altri tipi di modelli
    target_modules = ["query", "key", "value", "dense"]

print(f"Moduli target per LoRA: {target_modules}")

# Configurazione LoRA con rango aumentato
lora_config = LoraConfig(
    r=8,  # Aumentato da 4 a 8 per maggiore capacità di apprendimento
    lora_alpha=32,  # Aumentato proporzionalmente al rango
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.TOKEN_CLS
)

# Applica LoRA al modello
model = get_peft_model(model, lora_config)
print("PEFT/LoRA configurato con rango 8 per un migliore apprendimento")
print(f"Memoria GPU dopo setup LoRA: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Modifiche alla configurazione dell'addestramento per migliorare le performance
training_args = TrainingArguments(
    output_dir="./results_improved",
    eval_strategy="steps",           # Cambiato da "epoch" a "steps"
    eval_steps=100,                  # Valuta ogni 100 steps invece che ogni epoca
    save_strategy="steps",           # Cambiato da "epoch" a "steps"
    save_steps=100,                  # Salva ogni 100 steps invece che ogni epoca
    save_total_limit=3,              # Mantieni solo gli ultimi 3 checkpoint
    learning_rate=5e-5,
    per_device_train_batch_size=2,   # Aumentato da 1 a 2
    per_device_eval_batch_size=4,    # Aumentato da 1 a 4
    num_train_epochs=5,              # Ridotto da 15 a 5
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    gradient_accumulation_steps=16,  # Ridotto da 64 a 16
    dataloader_pin_memory=True,      # Cambiato a True per migliorare il trasferimento dati
    optim="adamw_8bit",
    fp16=True,
    torch_compile=True,              # Abilitato torch_compile per PyTorch 2.0+
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    no_cuda=False,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
)

# Crea un data collator specializzato per NER
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Definizione di un Trainer personalizzato estendendo la classe Trainer
class CustomTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.args.device) if class_weights is not None else None

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):  # Add num_items_in_batch
        # ... (rest of your compute_loss method remains the same) ...
        labels = inputs.pop("labels")
        outputs = model(**inputs)

        # Implementazione del class weighting
        if self.class_weights is not None:
            logits = outputs.logits
            active_loss = labels.view(-1) != -100
            active_logits = logits.view(-1, logits.shape[-1])[active_loss]
            active_labels = labels.view(-1)[active_loss]
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
            loss = loss_fct(active_logits, active_labels)
        else:
            # Default loss
            if isinstance(outputs, dict) and "loss" in outputs:
                loss = outputs["loss"]
            else:
                # Perdita standard per classificazione token
                loss_fct = torch.nn.CrossEntropyLoss()
                logits = outputs.logits
                active_loss = labels.view(-1) != -100
                active_logits = logits.view(-1, logits.shape[-1])[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)

        return (loss, outputs) if return_outputs else loss

# Verifica disponibilità di torch.compile
import torch
if hasattr(torch, "_dynamo") and torch.__version__ >= "2.0.0":
    print("torch.compile è disponibile e verrà utilizzato")
else:
    print("torch.compile non è disponibile, disabilitazione...")
    training_args.torch_compile = False

# Aggiunta delle callback al trainer
trainer = CustomTrainer(
    class_weights=class_weights_tensor,
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    callbacks=[TimingCallback(), MemoryMonitorCallback(print_every=100)],
)

# Libera ancora memoria prima dell'addestramento
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU prima dell'addestramento: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Avvio dell'addestramento
print("Avvio addestramento con 5 epoche...")
trainer.train()

# Salvataggio del modello addestrato
print("Salvataggio del modello addestrato...")
output_dir = "./model_ner_improved"
trainer.save_model(output_dir)
print(f"Modello migliorato salvato in: {output_dir}")

# Funzione per convertire le predizioni in etichette leggibili
def decode_predictions(preds, labels, id2label):
    decoded_preds = []
    decoded_labels = []

    for pred, label in zip(preds, labels):
        pred_tags = []
        label_tags = []

        for p, l in zip(pred, label):
            if l != -100:  # Ignora i token padding/speciali
                pred_tags.append(id2label[p])
                label_tags.append(id2label[l])

        decoded_preds.append(pred_tags)
        decoded_labels.append(label_tags)

    return decoded_preds, decoded_labels

# Valutazione sul set di test
print("\nValutazione sul set di test...")
predictions = trainer.predict(tokenized_datasets["test"])

# Estrai predizioni e etichette reali
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

# Converti le predizioni in formato leggibile
decoded_preds, decoded_labels = decode_predictions(preds, labels, id2label)

# Calcola e stampa le metriche principali
print("\nMetriche di valutazione:")
print(f"Precision: {precision_score(decoded_labels, decoded_preds):.4f}")
print(f"Recall: {recall_score(decoded_labels, decoded_preds):.4f}")
print(f"F1 Score: {f1_score(decoded_labels, decoded_preds):.4f}")

# Stampa il report di classificazione completo
print("\nReport di classificazione dettagliato:")
print(classification_report(decoded_labels, decoded_preds))

# Analisi delle predizioni su alcuni esempi
print("\nEsempi di predizioni:")
num_examples = min(5, len(decoded_preds))

# Ottieni i token originali per gli esempi
for i in range(num_examples):
    tokens = test_tokens[i]
    pred_tags = decoded_preds[i]
    true_tags = decoded_labels[i]

    # Assicuriamoci che le liste abbiano la stessa lunghezza
    min_len = min(len(tokens), len(pred_tags), len(true_tags))

    print(f"\nEsempio {i+1}:")
    print(f"{'Token':<15} | {'Etichetta vera':<15} | {'Etichetta predetta':<15}")
    print("-" * 50)

    for j in range(min_len):
        print(f"{tokens[j]:<15} | {true_tags[j]:<15} | {pred_tags[j]:<15}")

# Esempio di come utilizzare il modello per nuove predizioni
print("\nEsempio di come utilizzare il modello per nuove predizioni:")
print("""
# Funzione per predire entità in un nuovo testo
def predict_entities(text, model, tokenizer, id2label):
    # Tokenizza il testo
    tokens = text.split()
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")

    # Sposta gli input sulla GPU se disponibile
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # Ottieni le predizioni
    with torch.no_grad():
        outputs = model(**inputs)

    # Converti le predizioni in etichette
    predictions = outputs.logits.argmax(dim=-1)[0].cpu().numpy()

    # Converti gli ID delle predizioni in etichette
    word_ids = inputs.word_ids()[0]

    result = []
    previous_word_idx = None

    for idx, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx == previous_word_idx:
            continue

        pred_id = predictions[idx]
        pred_label = id2label[pred_id]
        result.append((tokens[word_idx], pred_label))

        previous_word_idx = word_idx

    return result

# Esempio di uso
example_text = "IL SINDACO MARIO ROSSI ha approvato la delibera del COMUNE DI MILANO"
entities = predict_entities(example_text, model, tokenizer, id2label)
print("Entità rilevate:")
for token, entity in entities:
    print(f"{token}: {entity}")
""")

# Implementazione effettiva della funzione predict_entities per test immediato
def predict_entities(text, model, tokenizer, id2label):
    # Tokenizza il testo
    tokens = text.split()
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")

    # Sposta gli input sulla GPU se disponibile
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # Ottieni le predizioni
    with torch.no_grad():
        outputs = model(**inputs)

    # Converti le predizioni in etichette
    predictions = outputs.logits.argmax(dim=-1)[0].cpu().numpy()

    # Converti gli ID delle predizioni in etichette
    word_ids = inputs.word_ids()[0]

    result = []
    previous_word_idx = None

    for idx, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx == previous_word_idx:
            continue

        pred_id = predictions[idx]
        pred_label = id2label[pred_id]
        result.append((tokens[word_idx], pred_label))

        previous_word_idx = word_idx

    return result

# Eseguiamo un test effettivo
try:
    print("\nTest di inferenza sul modello appena addestrato:")
    example_text = "IL SINDACO MARIO ROSSI ha approvato la delibera del COMUNE DI MILANO"
    entities = predict_entities(example_text, model, tokenizer, id2label)
    print("Entità rilevate:")
    for token, entity in entities:
        print(f"{token}: {entity}")
except Exception as e:
    print(f"Errore durante il test: {e}")
    print("Prova a caricare il modello salvato prima di eseguire l'inferenza.")

codice primo + secondo --> ottimizzato


In [ ]:
# Librerie necessarie
!pip install bitsandbytes accelerate -q
!pip install -q datasets
!pip install seqeval -q
!pip install deepspeed
import gc
import os
import torch
import numpy as np
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForTokenClassification, BitsAndBytesConfig, DataCollatorForTokenClassification
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training, PeftModel, PeftConfig
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers.integrations import TensorBoardCallback
import time
from transformers import TrainerCallback
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
import json

# Configurazione CUDA per evitare problemi di frammentazione memoria
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Per essere sicuri di liberare tutta la memoria disponibile
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU disponibile: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Memoria GPU utilizzata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Callback per monitorare tempi di addestramento
class TimingCallback(TrainerCallback):
    def __init__(self):
        self.start_time = time.time()
        self.step_start_time = time.time()
        self.steps = 0

    def on_step_end(self, args, state, control, **kwargs):
        self.steps += 1
        if self.steps % 10 == 0:  # Stampa ogni 10 steps
            elapsed = time.time() - self.step_start_time
            steps_per_second = 10 / elapsed
            print(f"Step {state.global_step}: {steps_per_second:.2f} steps/s, {elapsed/10:.3f} s/step")
            self.step_start_time = time.time()

    def on_epoch_end(self, args, state, control, **kwargs):
        elapsed = time.time() - self.start_time
        print(f"Epoca {state.epoch} completata. Tempo totale: {elapsed/60:.2f} minuti")

# Callback per monitorare l'utilizzo della memoria
class MemoryMonitorCallback(TrainerCallback):
    def __init__(self, print_every=100):
        self.print_every = print_every

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.print_every == 0:
            print(f"Step {state.global_step}")
            print(f"  Memoria allocata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
            print(f"  Memoria riservata: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
            print(f"  Memoria cache: {torch.cuda.memory_cached() / 1e9:.2f} GB")

# Funzione per pulire la memoria
def clean_memory():
    # Rilascia memoria Python
    gc.collect()

    # Rilascia memoria PyTorch
    torch.cuda.empty_cache()

    # Forza garbage collection
    import sys
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.device.type == 'cuda':
                print(f"Trovato tensor CUDA non referenziato: {obj.shape}")
                del obj
        except:
            pass

    gc.collect()
    torch.cuda.empty_cache()

    # Stampa memoria disponibile
    print(f"Memoria GPU disponibile: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Memoria GPU allocata: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Memoria GPU riservata: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

# Define the label_list, label2id, and id2label
with open("classes.txt", "r", encoding="utf-8") as f:
    label_list = [line.strip() for line in f.readlines()]
# Non aggiungiamo "O" poiché è già presente nel file classes.txt
#label_list = ["O"] + label_list
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}

print(f"Etichette caricate: {label_list}")
print(f"Mapping etichette -> ID: {label2id}")

# Define the load_iob_dataset function
def load_iob_dataset(path):
    sentences, labels = [], []
    with open(path, encoding='utf-8') as f:
        tokens, tags = [], []
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append(tokens)
                    labels.append(tags)
                    tokens, tags = [], []
            else:
                splits = line.split()
                if len(splits) >= 2:  # Assicurati che ci siano almeno due colonne
                    tokens.append(splits[0])
                    tags.append(splits[-1])
        if tokens:
            sentences.append(tokens)
            labels.append(tags)
    return sentences, labels

# Carica il tokenizer
model_name = "expertai/LLaMAntino-3-SLIMER-IT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Funzione ottimizzata per tokenizzare e allineare le etichette
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=32,              # Ridotto da 64 a 32 per migliorare performance
        return_tensors=None
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignora token speciali
            elif word_idx != previous_word_idx:
                # Solo il primo token di una parola ottiene l'etichetta
                # Gestione degli indici fuori limite
                if word_idx < len(label):
                    label_text = label[word_idx]
                else:
                    label_text = "O"  # Default a "O" per indici fuori limite
                label_id = label2id.get(label_text, label2id["O"])
                label_ids.append(label_id)
            else:
                # Token successivi della stessa parola
                label_ids.append(-100)  # Usiamo -100 per ignorare questi token nella loss

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Carica i dataset
print("Caricamento dei dataset...")
train_tokens, train_tags = load_iob_dataset("/content/train_sampled_iob_corretto.txt")
dev_tokens, dev_tags = load_iob_dataset("/content/dev_sampled_iob_corretto.txt")
test_tokens, test_tags = load_iob_dataset("/content/test_sampled_iob_corretto.txt")

# Analisi della lunghezza delle sequenze nel dataset
print("Analisi delle lunghezze delle sequenze:")
train_lengths = [len(seq) for seq in train_tokens]
print(f"Lunghezza media delle sequenze nel train set: {sum(train_lengths)/len(train_lengths):.1f} token")
print(f"Sequenza più lunga nel train set: {max(train_lengths)} token")
print(f"Percentile 95 delle lunghezze: {sorted(train_lengths)[int(len(train_lengths)*0.95)]} token")
print(f"Percentile 99 delle lunghezze: {sorted(train_lengths)[int(len(train_lengths)*0.99)]} token")

# Verifica la distribuzione delle etichette
print("Analisi della distribuzione delle etichette:")
all_tags = [tag for tag_list in train_tags for tag in tag_list]
unique_tags = set(all_tags)
print(f"Etichette uniche nel dataset: {unique_tags}")
tag_counts = {}
for tag in unique_tags:
    tag_counts[tag] = all_tags.count(tag)
    print(f"  - {tag}: {tag_counts[tag]} occorrenze")

# Calcolo pesi delle classi per class weighting
tag_weights = {}
total_tags = len(all_tags)
for tag in unique_tags:
    # Metodo inverso della frequenza
    tag_weights[tag] = total_tags / (tag_counts[tag] * len(unique_tags))
print(f"Pesi calcolati per le classi: {tag_weights}")

# Converti i pesi in un array per il modello
class_weights = []
for i in range(len(label_list)):
    label = id2label[i]
    if label in tag_weights:
        class_weights.append(tag_weights[label])
    else:
        # Gestisci etichette che potrebbero non essere presenti nei dati di training
        class_weights.append(1.0)

class_weights_tensor = torch.tensor(class_weights)
print(f"Tensor pesi classi: {class_weights_tensor}")

# Crea i dataset
train_ds = Dataset.from_dict({"tokens": train_tokens, "ner_tags": train_tags})
dev_ds = Dataset.from_dict({"tokens": dev_tokens, "ner_tags": dev_tags})
test_ds = Dataset.from_dict({"tokens": test_tokens, "ner_tags": test_tags})

dataset = DatasetDict({
    "train": train_ds,
    "validation": dev_ds,
    "test": test_ds
})

# Pre-processa i dataset
print("Pre-processing dei dataset...")
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["tokens", "ner_tags"]
)
print(f"Dataset processato: {len(tokenized_datasets['train'])} esempi nel train set")

# Verifica la struttura dei dati
print("Verifica della struttura dei dati:")
sample = tokenized_datasets["train"][0]
for key, value in sample.items():
    if isinstance(value, list):
        print(f"{key}: lista di lunghezza {len(value)}, tipo elementi: {type(value[0]).__name__}")
        if len(value) > 0 and isinstance(value[0], (int, float)):
            print(f"  - Esempio valori: {value[:5]}...")
    else:
        print(f"{key}: {type(value).__name__}")

# Esegui pulizia completa prima dell'addestramento
clean_memory()

# Configura la quantizzazione 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

print("Caricamento del modello con quantizzazione a 4-bit...")
# Miglior versione del caricamento del modello
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    device_map="auto",
    quantization_config=quantization_config,
    torch_dtype=torch.float16
)

# Monitora l'uso di memoria dopo il caricamento del modello
print(f"Memoria GPU dopo caricamento modello: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Prepara il modello per l'addestramento a 4-bit
model = prepare_model_for_kbit_training(model)

# Abilita checkpointing
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()
    print("Gradient checkpointing abilitato")

# Configurazione LoRA con moduli adatti al modello
# Trova quali moduli sono effettivamente nel modello
model_modules = [name for name, _ in model.named_modules()]
print("Moduli disponibili nel modello:")
for module in sorted(set([m.split('.')[-1] for m in model_modules if '.' in m]))[:15]:  # Mostra i nomi dei moduli unici
    print(f"  - {module}")
print("...")

# Scegli i moduli target in base a quelli disponibili
target_modules = []
for candidate in ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"]:
    if any(candidate in module for module in model_modules):
        target_modules.append(candidate)

if not target_modules:
    # Fallback per altri tipi di modelli
    target_modules = ["query", "key", "value", "dense"]

print(f"Moduli target per LoRA: {target_modules}")

# Configurazione LoRA con rango aumentato
lora_config = LoraConfig(
    r=8,  # Aumentato da 4 a 8 per maggiore capacità di apprendimento
    lora_alpha=32,  # Aumentato proporzionalmente al rango
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.TOKEN_CLS
)

# Applica LoRA al modello
model = get_peft_model(model, lora_config)
print("PEFT/LoRA configurato con rango 8 per un migliore apprendimento")
print(f"Memoria GPU dopo setup LoRA: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Modifiche alla configurazione dell'addestramento per migliorare le performance
training_args = TrainingArguments(
    output_dir="./results_improved",
    eval_strategy="steps",           # Cambiato da "epoch" a "steps"
    eval_steps=100,                  # Valuta ogni 100 steps invece che ogni epoca
    save_strategy="steps",           # Cambiato da "epoch" a "steps"
    save_steps=100,                  # Salva ogni 100 steps invece che ogni epoca
    save_total_limit=3,              # Mantieni solo gli ultimi 3 checkpoint
    learning_rate=5e-5,
    per_device_train_batch_size=2,   # Aumentato da 1 a 2
    per_device_eval_batch_size=4,    # Aumentato da 1 a 4
    num_train_epochs=15,              # Ridotto da 15 a 5
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    gradient_accumulation_steps=16,  # Ridotto da 64 a 16
    dataloader_pin_memory=True,      # Cambiato a True per migliorare il trasferimento dati
    optim="adamw_8bit",
    fp16=True,
    torch_compile=True,              # Abilitato torch_compile per PyTorch 2.0+
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    no_cuda=False,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
)

# Crea un data collator specializzato per NER
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Definizione di un Trainer personalizzato estendendo la classe Trainer
class CustomTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.args.device) if class_weights is not None else None

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)

        # Implementazione del class weighting
        if self.class_weights is not None:
            logits = outputs.logits
            active_loss = labels.view(-1) != -100
            active_logits = logits.view(-1, logits.shape[-1])[active_loss]
            active_labels = labels.view(-1)[active_loss]
            loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights)
            loss = loss_fct(active_logits, active_labels)
        else:
            # Default loss
            if isinstance(outputs, dict) and "loss" in outputs:
                loss = outputs["loss"]
            else:
                # Perdita standard per classificazione token
                loss_fct = torch.nn.CrossEntropyLoss()
                logits = outputs.logits
                active_loss = labels.view(-1) != -100
                active_logits = logits.view(-1, logits.shape[-1])[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)

        return (loss, outputs) if return_outputs else loss

# Verifica disponibilità di torch.compile
import torch
if hasattr(torch, "_dynamo") and torch.__version__ >= "2.0.0":
    print("torch.compile è disponibile e verrà utilizzato")
else:
    print("torch.compile non è disponibile, disabilitazione...")
    training_args.torch_compile = False

# Aggiunta delle callback al trainer (con MemoryMonitorCallback)
trainer = CustomTrainer(
    class_weights=class_weights_tensor,
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    callbacks=[TimingCallback(), MemoryMonitorCallback(print_every=100)],
)

# Libera ancora memoria prima dell'addestramento
gc.collect()
torch.cuda.empty_cache()
print(f"Memoria GPU prima dell'addestramento: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Avvio dell'addestramento
print("Avvio addestramento con 5 epoche...")
trainer.train()

# Salvataggio del modello addestrato
print("Salvataggio del modello addestrato...")
output_dir = "./model_ner_improved"
trainer.save_model(output_dir)
print(f"Modello migliorato salvato in: {output_dir}")

# Salva anche le mappature di etichette in un file JSON per uso futuro
with open(os.path.join(output_dir, "label_maps.json"), "w", encoding="utf-8") as f:
    json.dump({"id2label": id2label, "label2id": label2id}, f, ensure_ascii=False, indent=2)

# Funzione per convertire le predizioni in etichette leggibili
def decode_predictions(preds, labels, id2label):
    decoded_preds = []
    decoded_labels = []

    for pred, label in zip(preds, labels):
        pred_tags = []
        label_tags = []

        for p, l in zip(pred, label):
            if l != -100:  # Ignora i token padding/speciali
                pred_tags.append(id2label[p])
                label_tags.append(id2label[l])

        decoded_preds.append(pred_tags)
        decoded_labels.append(label_tags)

    return decoded_preds, decoded_labels

# Valutazione sul set di test
print("\nValutazione sul set di test...")
predictions = trainer.predict(tokenized_datasets["test"])

# Estrai predizioni e etichette reali
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

# Converti le predizioni in formato leggibile
decoded_preds, decoded_labels = decode_predictions(preds, labels, id2label)

# Calcola e stampa le metriche principali
print("\nMetriche di valutazione:")
print(f"Precision: {precision_score(decoded_labels, decoded_preds):.4f}")
print(f"Recall: {recall_score(decoded_labels, decoded_preds):.4f}")
print(f"F1 Score: {f1_score(decoded_labels, decoded_preds):.4f}")

# Stampa il report di classificazione completo
print("\nReport di classificazione dettagliato:")
print(classification_report(decoded_labels, decoded_preds))

# Visualizzazione di esempi di predizioni dal test set
print("\nEsempi di predizioni dal test set:")
num_examples = min(5, len(decoded_preds))

# Ottieni i token originali per gli esempi
for i in range(num_examples):
    tokens = test_tokens[i]
    pred_tags = decoded_preds[i]
    true_tags = decoded_labels[i]

    # Assicuriamoci che le liste abbiano la stessa lunghezza
    min_len = min(len(tokens), len(pred_tags), len(true_tags))

    print(f"\nEsempio {i+1}:")
    print(f"{'Token':<15} | {'Etichetta vera':<15} | {'Etichetta predetta':<15}")
    print("-" * 50)

    for j in range(min_len):
        print(f"{tokens[j]:<15} | {true_tags[j]:<15} | {pred_tags[j]:<15}")

# Funzione completa per predire entità in un nuovo testo
def predict_entities(text, model, tokenizer, id2label):
    # Tokenizza il testo
    tokens = text.split()
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")

    # Sposta gli input sulla GPU se disponibile
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # Ottieni le predizioni
    with torch.no_grad():
        outputs = model(**inputs)

    # Converti le predizioni in etichette
    predictions = outputs.logits.argmax(dim=-1)[0].cpu().numpy()

    # Converti gli ID delle predizioni in etichette
    word_ids = inputs.word_ids()[0]

    result = []
    previous_word_idx = None

    for idx, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx == previous_word_idx:
            continue

        if idx < len(predictions):  # Verifica indice fuori limite
            pred_id = predictions[idx]
            if pred_id in id2label:  # Verifica che l'ID sia valido
                pred_label = id2label[pred_id]
                if word_idx < len(tokens):  # Verifica indice fuori limite
                    result.append((tokens[word_idx], pred_label))

        previous_word_idx = word_idx

    return result

# Sezione migliorata di test su esempi specifici
print("\n" + "="*50)
print("TEST DEL MODELLO SU ESEMPI SPECIFICI")
print("="*50)

# Lista di esempi per testare il modello
test_examples = [
    "IL SINDACO MARIO ROSSI ha approvato la delibera del COMUNE DI MILANO",
    "La REGIONE LOMBARDIA ha stanziato fondi per la PROVINCIA DI COMO",
    "Il DECRETO LEGISLATIVO n.50 del 2016 ha modificato il CODICE DEGLI APPALTI",
    "Il TRIBUNALE DI ROMA ha emesso una sentenza riguardante il caso di GIUSEPPE VERDI",
    "L'AGENZIA DELLE ENTRATE ha pubblicato una circolare sull'applicazione della LEGGE 104"
]

# Test di inferenza con gestione degli errori
try:
    print("\nTest di inferenza sul modello appena addestrato:")

    for i, example_text in enumerate(test_examples):
        print(f"\nEsempio {i+1}: \"{example_text}\"")
        entities = predict_entities(example_text, model, tokenizer, id2label)

        # Organizza le entità per tipo
        entities_by_type = {}
        for token, entity in entities:
            if entity != "O":  # Escludiamo le entità di tipo "O" (non-entità)
                if entity not in entities_by_type:
                    entities_by_type[entity] = []
                entities_by_type[entity].append(token)

        # Stampa il risultato in modo organizzato
        if entities_by_type:
            print("Entità rilevate:")
            for entity_type, tokens in entities_by_type.items():
                print(f"  {entity_type}: {', '.join(tokens)}")
        else:
            print("Nessuna entità rilevata in questo esempio.")

        # Visualizza in formato evidenziato (simulato tramite testo)
        print("\nVisualizziamo le entità nel testo:")
        tokens = example_text.split()
        highlighted_text = []

        for token in tokens:
            found = False
            for entity_token, entity_type in entities:
                if token == entity_token and entity_type != "O":
                    highlighted_text.append(f"[{token}]_{entity_type}")
                    found = True
                    break
            if not found:
                highlighted_text.append(token)

        print(" ".join(highlighted_text))
        print("-"*50)

    # Prova ad analizzare un testo più lungo
    print("\nTest su un testo più lungo e complesso:")
    long_text = """
    Il CONSIGLIO COMUNALE DI MILANO, presieduto dal SINDACO MARIO ROSSI,
    ha approvato la nuova DELIBERA 123/2023 relativa all'applicazione del
    DECRETO LEGISLATIVO 267/2000. Durante la seduta, sono intervenuti
    anche rappresentanti della REGIONE LOMBARDIA e dell'AGENZIA DELLE ENTRATE.
    La discussione ha riguardato anche il rispetto della COSTITUZIONE ITALIANA
    e delle normative europee emanate dal PARLAMENTO EUROPEO.
    """

    # Pulisci il testo lungo
    long_text = " ".join(long_text.strip().split())
    print(f"Testo: \"{long_text}\"")

    # Dividi il testo in parti più piccole per evitare problemi di memoria
    chunk_size = 20  # Numero di token per chunk
    tokens = long_text.split()
    chunks = [" ".join(tokens[i:i+chunk_size]) for i in range(0, len(tokens), chunk_size)]

    all_entities = []
    for chunk in chunks:
        entities = predict_entities(chunk, model, tokenizer, id2label)
        all_entities.extend(entities)

    # Organizza le entità per tipo
    entities_by_type = {}
    for token, entity in all_entities:
        if entity != "O":  # Escludiamo le entità di tipo "O" (non-entità)
            if entity not in entities_by_type:
                entities_by_type[entity] = []
            entities_by_type[entity].append(token)

    # Stampa il risultato in modo organizzato
    if entities_by_type:
        print("\nEntità rilevate nel testo lungo:")
        for entity_type, tokens in entities_by_type.items():
            print(f"  {entity_type}: {', '.join(tokens)}")
    else:
        print("Nessuna entità rilevata nel testo lungo.")

except Exception as e:
    print(f"Errore durante il test di inferenza: {e}")
    print("Prova a caricare il modello salvato prima di eseguire l'inferenza.")

# Codice per inferenza con modello salvato (da eseguire separatamente se necessario)
def load_and_test_saved_model():
    try:
        print("\nCaricamento del modello salvato per inferenza...")
        # Carica la configurazione delle etichette
        with open(os.path.join("./model_ner_improved", "label_maps.json"), "r") as f:
            label_maps = json.load(f)
            saved_id2label = {int(k): v for k, v in label_maps["id2label"].items()}

        # Carica il modello salvato
        saved_model = PeftModel.from_pretrained(
            AutoModelForTokenClassification.from_pretrained(
                model_name,
                device_map="auto",
                torch_dtype=torch.float16
            ),
            "./model_ner_improved",
            device_map="auto"
        )
        saved_model.eval()

        # Test su un esempio
        example_text = "I cittadini di ROMA e MILANO hanno votato per il SINDACO Roberto ROSSI"
        entities = predict_entities(example_text, saved_model, tokenizer, saved_id2label)

        print(f"Testo: {example_text}")
        print("Entità rilevate:")
        for token, entity in entities:
            if entity != "O":
                print(f"{token}: {entity}")

        return True
    except Exception as e:
        print(f"Errore nel caricamento del modello salvato: {e}")
        return False

# Mostra come utilizzare il modello per inferenza in futuro
print("\nCodice per utilizzare il modello in futuro:")
print("""
# Per utilizzare il modello salvato in futuro:
from transformers import AutoModelForTokenClassification, AutoTokenizer
from peft import PeftModel
import torch
import json

# Carica il tokenizer
tokenizer = AutoTokenizer.from_pretrained("expertai/LLaMAntino-3-SLIMER-IT")

# Carica le mappature delle etichette
with open("./model_ner_improved/label_maps.json", "r") as f:
    label_maps = json.load(f)
    id2label = {int(k): v for k, v in label_maps["id2label"].items()}

# Carica il modello base e l'adattatore LoRA
base_model = AutoModelForTokenClassification.from_pretrained(
    "expertai/LLaMAntino-3-SLIMER-IT",
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, "./model_ner_improved")
model.eval()

# Funzione di inferenza
def predict_entities(text):
    tokens = text.split()
    inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")

    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = outputs.logits.argmax(dim=-1)[0].cpu().numpy()
    word_ids = inputs.word_ids()[0]

    result = []
    previous_word_idx = None

    for idx, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx == previous_word_idx:
            continue

        pred_id = predictions[idx]
        pred_label = id2label[pred_id]

        if pred_label != "O":  # Mostra solo le entità non-O
            result.append((tokens[word_idx], pred_label))

        previous_word_idx = word_idx

    return result

# Esempio di utilizzo
text = "La riunione del CONSIGLIO COMUNALE di MILANO si terrà domani"
entities = predict_entities(text)
for token, entity in entities:
    print(f"{token}: {entity}")
""")

print("\nAddestramento e valutazione del modello completati con successo!")

Memoria GPU disponibile: 15.83 GB
Memoria GPU utilizzata: 0.00 GB
Etichette caricate: ['O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-LAW', 'I-LAW', 'B-ACT', 'I-ACT', 'B-ORG', 'I-ORG', 'B-OPA', 'I-OPA']
Mapping etichette -> ID: {'O': 0, 'B-LOC': 1, 'I-LOC': 2, 'B-PER': 3, 'I-PER': 4, 'B-LAW': 5, 'I-LAW': 6, 'B-ACT': 7, 'I-ACT': 8, 'B-ORG': 9, 'I-ORG': 10, 'B-OPA': 11, 'I-OPA': 12}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Caricamento dei dataset...
Analisi delle lunghezze delle sequenze:
Lunghezza media delle sequenze nel train set: 29.2 token
Sequenza più lunga nel train set: 246 token
Percentile 95 delle lunghezze: 86 token
Percentile 99 delle lunghezze: 182 token
Analisi della distribuzione delle etichette:
Etichette uniche nel dataset: {'I-PER', 'I-LOC', 'I-ORG', 'I-OPA', 'O', 'I-LAW', 'I-ACT'}
  - I-PER: 184 occorrenze
  - I-LOC: 298 occorrenze
  - I-ORG: 215 occorrenze
  - I-OPA: 165 occorrenze
  - O: 6736 occorrenze
  - I-LAW: 866 occorrenze
  - I-ACT: 246 occorrenze
Pesi calcolati per le classi: {'I-PER': 6.762422360248447, 'I-LOC': 4.175455417066155, 'I-ORG': 5.787375415282392, 'I-OPA': 7.541125541125541, 'O': 0.18472175093315235, 'I-LAW': 1.4368195315077532, 'I-ACT': 5.0580720092915215}
Tensor pesi classi: tensor([0.1847, 1.0000, 4.1755, 1.0000, 6.7624, 1.0000, 1.4368, 1.0000, 5.0581,
        1.0000, 5.7874, 1.0000, 7.5411])
Pre-processing dei dataset...


Map:   0%|          | 0/298 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

...

No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Memoria GPU prima dell'addestramento: 5.79 GB
Avvio addestramento con 5 epoche...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


Step 10: 0.05 steps/s, 21.632 s/step
Epoca 1.0 completata. Tempo totale: 3.61 minuti
Step 20: 0.05 steps/s, 20.500 s/step
Epoca 2.0 completata. Tempo totale: 7.02 minuti
Step 30: 0.05 steps/s, 20.516 s/step
Epoca 3.0 completata. Tempo totale: 10.44 minuti
Step 40: 0.05 steps/s, 20.523 s/step
Epoca 4.0 completata. Tempo totale: 13.86 minuti
Step 50: 0.05 steps/s, 20.567 s/step
Epoca 5.0 completata. Tempo totale: 17.29 minuti
Step 60: 0.05 steps/s, 20.524 s/step
Epoca 6.0 completata. Tempo totale: 20.71 minuti
Step 70: 0.05 steps/s, 20.512 s/step
Epoca 7.0 completata. Tempo totale: 24.13 minuti
Step 80: 0.05 steps/s, 20.508 s/step
Epoca 8.0 completata. Tempo totale: 27.55 minuti
Step 90: 0.05 steps/s, 20.496 s/step
Epoca 9.0 completata. Tempo totale: 30.96 minuti
